# imports

In [ ]:
%matplotlib inline

In [ ]:
from IPython.core import debugger as idb

In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
import numpy as np

In [ ]:
# export
from fastai.vision import *

In [ ]:
import os

In [ ]:
import cv2

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import random

# functions

## 缓存图片，加速训练

应在split之后再执行cache. 因为，若在split之前的ItemList执行cache，则在下一步split时，会新建train和valid这两个ItemList，而新建的ItemList是不带有cached_images属性的，那么你在split之前执行的cache就不起作用了。  
理论上说，你可以在split之后的任何一处执行cache，例如对ItemLists的train和valid这两个ItemList分别执行cache_image，但是我们建议在databunch之后再执行cache_image，而且为了方便此操作而定义了Databunch.cache_ds_image()方法。  
这样建议的原因是：  
- 在逻辑上更合理。缓存图片的目的是加速训练过程中的图片加载速度，而为模型提供训练数据的是databunch，那么在databunch处执行缓存就在流程上更加清晰。
- 在资源占用上更合理。一旦我们执行了缓存图片，那么就占用了大量的内存，后续的程序执行中可用内存就更少了。所以我们不应该提早的把这个内存占用起来，而应该在最后时刻再占用，这个时刻就是训练即将开始时。

In [ ]:
# export
# 为ImageList类添加cache_image方法
def cache_image(self):
    cached_images = [self.open(fn) for fn in self.items]
    self.cached_images = cached_images

ImageList.cache_image = cache_image

In [ ]:
# export
# 修改ImageList类的get方法
def get(self,i):
    if hasattr(self, 'cached_images'):
        res = self.cached_images[i]
    else:
        fn = super(ImageList, self).get(i)
        res = self.open(fn)
        
    self.sizes[i] = res.size
    return res

ImageList.get = get

In [ ]:
# export
# 为 Databunch类添加cache_ds_img方法
def cache_ds_img(self):
    self.train_ds.x.cache_image()
    self.valid_ds.x.cache_image()
    
ImageDataBunch.cache_ds_img = cache_ds_img

## get_label_from_df 函数

In [ ]:
#export
pat_coord = re.compile(r'\d+')
pat_clas = re.compile(r'\w+')
pat_imgName = re.compile(r'(\w+/\d+\.png)$')
kkk = 0
def get_label_from_df(fn, df, pat_imgName, box_col, cat_col):
    '''
    fn: 
        file path.
    df: 
        a dataframe stores all the label information, imageName shoud be as index.
    repat_imgName: 
        a regular expression pattern, used to find the imageName from fn, where imageName is stored in df 
    box_col:
        the column name of bounding boxs
    cat_col:
        the column name of categories
    '''
    pat_num = re.compile(r'\d+')
    pat_cat = re.compile(r'\w+')
    
    fn = pat_imgName.findall(fn)[0]
    
    boxes = df.loc[fn,box_col]
    boxes = pat_num.findall(boxes)
    boxes = list(map(np.long, boxes))
    boxes = np.array(boxes).reshape(-1,4)
    boxes = boxes.tolist()
    
    cats = df.loc[fn,cat_col]
    cats = pat_clas.findall(cats)
    
    assert len(boxes)==len(cats), 'length of bounding boxes and categories not equeal.'
        
    return (boxes,cats)

## 修改 ImageBBox.create()

使可以表示“无目标”。  
在目标检测任务中，训练集中可以有无目标的训练样本，其价值在于告诉模型这些是“我们不需要的”。但是在fastai的目标检测中是不支持无目标的训练样本的。  
为了增加对“无目标”样本的支持，需要考虑怎样表示“无目标”的标注信息。fastai中以ImageBBox来表示目标检测任务中的标注信息，


In [ ]:
# export
@classmethod
def create(cls, h:int, w:int, bboxes:Collection[Collection[int]], labels:Collection=None, classes:dict=None,
           pad_idx:int=0, scale:bool=True)->'ImageBBox':
    "Create an ImageBBox object from `bboxes`."
    # the following 3 lines are added by xutongye
#     assert isinstance(bboxes,list) and isinstance(labels,list), 'bboxes and labels should be of type list.'
    if isinstance(bboxes,list) and len(bboxes)==0:
        bboxes = [[0,0,0,0]]
    # the code in fastai
    if isinstance(bboxes, np.ndarray) and bboxes.dtype == np.object: bboxes = np.array([bb for bb in bboxes])
    bboxes = tensor(bboxes).float()
    tr_corners = torch.cat([bboxes[:,0][:,None], bboxes[:,3][:,None]], 1)
    bl_corners = bboxes[:,1:3].flip(1)
    bboxes = torch.cat([bboxes[:,:2], tr_corners, bl_corners, bboxes[:,2:]], 1)
    flow = FlowField((h,w), bboxes.view(-1,2))
    return cls(flow, labels=labels, classes=classes, pad_idx=pad_idx, y_first=True, scale=scale)

ImageBBox.create = create

## 修改 ImageBBox._compute_boxes()

In [ ]:
# export
def _compute_boxes(self) -> Tuple[LongTensor, LongTensor]:
    '''
    Check if there are bad bboxes (boxes whose area is zero). If there are, delete them.
    Bad boxes may be due to transformations, for example may rotate a bbox out of the image.
    Bad boxes may be added in the initial, because there indeed no object, but you has to add one, or there will be confilicts occur if you dont. 
    '''
    bboxes = self.flow.flow.flip(1).view(-1, 4, 2).contiguous().clamp(min=-1, max=1)
    mins, maxes = bboxes.min(dim=1)[0], bboxes.max(dim=1)[0]
    bboxes = torch.cat([mins, maxes], 1)
    mask = (bboxes[:,2]-bboxes[:,0] > 0) * (bboxes[:,3]-bboxes[:,1] > 0)
    #if len(mask) == 0: return tensor([self.pad_idx] * 4), tensor([self.pad_idx])
    # 上句判断 len(mask)==0 是个bug
    if mask.sum()==0: 
        return tensor([[self.pad_idx]*4]), tensor([self.pad_idx])
    res = bboxes[mask]
    if self.labels is None: return res,None
    return res, self.labels[to_np(mask).astype(bool)]

ImageBBox._compute_boxes = _compute_boxes

## 增加transform操作：rot90_affine

(该函数引用自kechan的[分享](https://github.com/kechan/FastaiPlayground))

注意：在fastai中，flip_lr 和 dihedral 有 tfmpixel 和 tfmaffine（flip_affine,dihedral_affine)两种实现方式。
- tfmpixel方式是直接对图像数据（tensor）的维度做操作；
- tfmaffine则是通过(i)create coordinate grid; (ii)affine matrix multiplicatioin; (iii)interpolation的方式来实现（见[该说明](https://forums.fast.ai/t/new-coordinate-transforms-pipeline/19790)）。   

对于flip_lr和dihedral两种操作来说，tfmpixel实现方式的计算量更小，但是它仅能对Image做操作，而不能用于ImagePoints和ImageBBox. tfmaffine则更具有通用性.  

此处实现的 rot90 操作也可以 tfmpixel 和 tfmaffine 来实现，而因为我们这里要对 Image 和 ImageBBox 都做该操作，所以我们以 tfmaffine 的方式实现。

In [ ]:
# export
def _rot90_affine(k:partial(uniform_int, 0, 3)):
    "Randomly rotate `x` image based on `k` as in np.rot90"
    if k%2 == 0:
        x = -1. if k&2 else 1.
        y = -1. if k&2 else 1.
        
        return [[x, 0, 0.],
                [0, y, 0],
                [0, 0, 1.]]
    else:
        x = 1. if k&2 else -1.
        y = -1. if k&2 else 1.
        
        return [[0, x, 0.],
                [y, 0, 0],
                [0, 0, 1.]]

rot90_affine = TfmAffine(_rot90_affine)

## get_stats

In [ ]:
# export
def get_stats(data,dl_types=[DatasetType.Train]):
    '''
    根据一个databunch中的所有指定的（图片）数据来统计各通道的均值和标准差。
    --------------------------------
    参数：
    -- data：一个DataBunch对象
    -- dl_types：一个list，其元素可选自DatasetType.Train, DatasetType.Valid, DatasetType.Test
    --------------------------------
    返回值：
    -- mean：图片三个通道上的均值
    -- std：图片三个通道上的标准差
    '''
    tn,sm,ssm = 0,torch.zeros(3,device=data.device),torch.zeros(3,device=data.device) # tn: total number; sm: sum; ssm: square sum
    for dl_type in dl_types:
        dl = data.dl(dl_type)
        for x,_ in dl:
            tn += x.shape[0]
            sm += x.mean((0,2,3))*x.shape[0]
            ssm += x.pow(2).mean((0,2,3))*x.shape[0]
            
    mean = sm/tn
    std = (ssm/tn - mean.pow(2)).sqrt()
    
    return mean, std

## train和valid数据统计

In [ ]:
# export
def databunch_statistics(data:DataBunch,show=True):
    train_ys = data.train_ds.y
    valid_ys = data.valid_ds.y
    
    hw_statistics = [[],[]]
    cat_statistics = [dict([(cla,0) for cla in data.train_dl.y.classes[1:]]),
                      dict([(cla,0) for cla in data.train_dl.y.classes[1:]])]
    
    for hw_stat,cat_stat,ys in zip(hw_statistics, cat_statistics, [train_ys,valid_ys]):
        for y in ys:
            if len(y.labels)==0: continue

            # 统计目标类别
            for clas in y.labels:
                cat_stat[clas.obj] += 1

            # 统计高宽
            pts = y.flow.flow
            pts = pts.reshape(-1,4,2)
            hws = pts.max(dim=1)[0] - pts.min(dim=1)[0]
            hw_stat += [hws]
            
    train_cat,valid_cat = cat_statistics
    train_hw,valid_hw = hw_statistics
    train_hw = torch.cat(train_hw,dim=0)
    valid_hw = torch.cat(valid_hw,dim=0)
    
    if show:
        
        
        # 显示各类别计数
        print('{:>41}{:>10}'.format('train','valid'))
        print('------------------------------------------------------')
        train_tot = 0
        valid_tot = 0
        for k,v in train_cat.items():
            valid_v = valid_cat[k]
            print('{:>30}:{:>10}{:>10}'.format(k,v,valid_v))
            train_tot += v
            valid_tot += valid_v
        print('------------------------------------------------------')
        print('{:>30}:{:>10}{:>10}'.format('total',train_tot,valid_tot))
        
        # 绘制高宽分布
        print('\n\n')
        print('red for train; blue for valid:')
        plt.scatter(train_hw[:,0],train_hw[:,1],c='r',marker='.',linewidths=2);
        plt.scatter(valid_hw[:,0],valid_hw[:,1],c='b',marker='.',linewidths=0);

    
    return train_hw, valid_hw, train_cat, valid_cat

# test

In [ ]:
# 做些设置
# data_root = './data/tiny_ds_20200331/'
data_root = './data/ds_20200227/'
data_root = Path(data_root)

csv_name = 'gends.csv'
csv_path = data_root/csv_name

img_subpath = 'images'
img_path = data_root/img_subpath

bs = 64

device = 'cpu'
device = torch.device('cuda')

In [ ]:
# 读入csv，稍作处理，方便get_label函数操作
df = pd.read_csv(csv_path,index_col=0)
df = df.set_index('image')
df.head()

In [ ]:
# ItemList
data = ObjectItemList.from_csv(path=data_root, csv_name=csv_name, cols='image')

In [ ]:
# split ItemList to get ItemLists
data = data.split_by_rand_pct(valid_pct=0.2)

In [ ]:
# label ItemLists to get LabelLists
pat_imgName = re.compile(r'(\w+/\d+\.jpg)$')
func = partial(get_label_from_df, df=df, pat_imgName=pat_imgName, box_col='box', cat_col='cls')
data = data.label_from_func(func=func)

In [ ]:
# add transforms
trn_tfms = [*zoom_crop(scale=(0.9,1.1),do_rand=True,p=1),
            rot90_affine(use_on_y=True)]
val_tfms = []

data = data.transform(tfms=[trn_tfms,val_tfms], tfm_y=True, remove_out=True)

**关于 *add transforms***  
  
**1，添加哪些transform？**  
- 缩放：同一类符号是否会出现缩放上的不同？这还需要对数据做统计观察。
- 90,180,270度旋转：我们观察到了同一类符号会有这类旋转。  

**2，为什么zoom_crop前加 \* 符号？**  
LabelLists.transform的参数tfms必须满足形式[[Tfm,Tfm,...],[Tfm,Tfm,...]]，（其中Tfm表示一个Transform对象）。而zoom_crop返回的是一个list：[Tfm,Tfm]，通过前加\*对list解包。
  
**3，关于transform的remove_out参数：**  
从fastai的源码看，remove_out起作用在image.py->ImagePoints.data函数中调用_remove_points_out 函数：只要有超过[-1,1]的点，它就把它删掉。但调试发现，不论remove_out为True或False，都没有调用到_remove_points_out，所以没有搞清楚remove_out到底如何能起作用。试验发现，不论remove_out为True或False，其现象都一样，经过transform（例如放大）后：
- 若bbox的包围区域整个落在了边界之外，则它被删除；
- 只要bbox的包围区域有部分落在边界之内，则它被保留，但会被切割在[-1,1]范围内。

**4，ds_tfms 和 dl_tfms**  
ds_tfms指添加在dataset上的transform，dl_tfms指添加在deviceDataloader（fastai的类）上的transform. dl_tfms的优点是可以在GPU上按batch做处理，大大提高效率。但是目前就我从fastai-v1的源码来看，其对dl_tfms的支持还相对较少，基本需要完全自己编写dl_tfms函数。从时间上考虑，我们暂时选择都用 ds_tfms，后续可以改为使用 dl_tfms，或者 fastai-v2 对 dl_tfms 有更完善的支持。

In [ ]:
# create DataBunch from LabelLists
data = data.databunch(bs=bs, device=device, collate_fn=bb_pad_collate)

In [ ]:
# 归一化
# 当bs较大时，可以不必传入stats参数，这是normalize函数内部会取data的一个batch来统计stats，并以此近似整个数据集的stats
# 但是当bs较小时，这样做就很不准确，需要先统计整个数据集的stats，在这里作为参数传入
data = data.normalize()

In [ ]:
# 缓存图片
# data.cache_ds_img()

In [ ]:
data.show_batch(rows=3)

In [ ]:
tdl = data.train_dl

In [ ]:
tdldl = tdl.dl

In [ ]:
tdldl.sampler

In [ ]:
# 查看统计信息
databunch_statistics(data);

## zip process as a function

将process组中的过程打包为函数，方便其它模块快速获取结果的函数。该组下的函数一般都是为了测试用，而非通用。因为在设计这些函数时，更多的考虑是调用简单，而不会仔细考虑其灵活性和合理性。

In [ ]:
# export
# 这个函数是为了在其它模块的设计时快速构造databunch
def get_databunch(data_root='./data/ds_20200227', csv_name='gends.csv', valid_pct=0.2, bs=64, device=torch.device('cpu'), cache=False):
    '''
    --------------------------------
    参数：
    -- data_root：数据集的总目录
    -- csv_name：存放标注信息的csv文件名，其要符合“对csv的要求”
    -- valid_pct：随机分割训练/验证集，该参数指定验证集的比例
    -- bs：batch size
    -- device：在datalaoder迭代时，dataloader先将batch加载到该device，做batch transform，然后返回。
    -- cache：dataset是否将所有图片预缓存入内存
    --------------------------------
    返回值：
    -- 一个databunch对象
    --------------------------------
    对csv的要求：
    1，带index
    2，存放图片名的列名称为"image"
    3，存放bbox信息的列名称为"box"
    4，存放类别信息的列名称为"cls"
    --------------------------------
    '''
    data_root = Path(data_root)
    csv_name = Path('gends.csv')
    # 读入csv，稍作处理，方便get_label函数操作
    csv_path = data_root/csv_name
    df = pd.read_csv(csv_path,index_col=0)
    df = df.set_index('image')

    # ItemList
    data = ObjectItemList.from_csv(path=data_root, csv_name=csv_name, cols='image')

    # split ItemList to get ItemLists
    data = data.split_by_rand_pct(valid_pct=valid_pct)

    # label ItemLists to get LabelLists
    pat_imgName = re.compile(r'(\w+/\d+\.jpg)$')
    func = partial(get_label_from_df, df=df, pat_imgName=pat_imgName, box_col='box', cat_col='cls')
    data = data.label_from_func(func=func)

    # add transforms
    trn_tfms = [*zoom_crop(scale=(0.9,1.1),do_rand=True,p=1),
                rot90_affine(use_on_y=True)]
    val_tfms = []

    data = data.transform(tfms=[trn_tfms,val_tfms], tfm_y=True, remove_out=True)

    # create DataBunch from LabelLists
    data = data.databunch(bs=bs, device=device, collate_fn=bb_pad_collate, num_workers=0)

    # normalize
    data = data.normalize()
    
    # 缓存图片
    if cache:
        data.cache_ds_img()
        
    return data

# export

In [2]:
!python ../../notebook2script.py --fname 'databunch.ipynb' --outputDir '../exp/'

Converted databunch.ipynb to ../exp/databunch.py
